# In this script we are going to process our fmri data with nipype 

## First we need to import our necessary modules and assign the SPM and matlab environment.

In [8]:
%matplotlib inline
from os.path import join as opj
import os
import json
from nipype.interfaces.spm import Smooth, Realign, Normalize12, SliceTiming
from nipype.interfaces.utility import IdentityInterface
from nipype.interfaces.io import SelectFiles, DataSink
from nipype import Workflow, Node
from nipype.algorithms.misc import Gunzip
from nipype.interfaces import spm
matlab_cmd = '/home/jt/temp/spm12_latest/spm12/run_spm12.sh /home/jt/Matlab/v98 script'
spm.SPMCommand.set_mlab_paths(matlab_cmd=matlab_cmd, use_mcr=True)

## Define your paths of BIDS

In [9]:
experiment_dir = '/home/jt/ds002748'
output_dir = '/media/jt/Daten1/preproc/datasink'
working_dir = '/media/jt/Daten1/preproc/workingdir'

## Now we are choosing our subjects and the smoothing intensity.

In [10]:
# list of subject identifiers
subject_list = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', 
               '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', 
               '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', 
               '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', 
               '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', 
               '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', 
               '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', 
               '71', '72']

# list of session identifiers
task_list = ['resting state with closed eyes']

# Smoothing widths to apply
fwhm = [8]

# TR of functional images
with open('/home/jt/ds002748/task-rest_bold.json', 'rt') as fp:
    task_info = json.load(fp)
TR = task_info['RepetitionTime']

# Isometric resample of functional images to voxel size (in mm)
iso_size = 4

## Define our different nodes we want to process during the script. In the original paper they made 4 steps: normalize, realign, slice timing and smoothing. So we are doing here the same + gunzip to unzip our data.

In [11]:
realign = Node(Realign(register_to_mean=True), name="realign")
norm12 = Node(spm.Normalize12(), name="normalise")
st = Node(SliceTiming(), name="slicetiming")
st.inputs.num_slices=25
st.inputs.ref_slice=1
st.inputs.time_repetition=2.5
st.inputs.slice_order=list(range(0,25,+1))
st.inputs.time_acquisition=(2.4)

smooth = Node(Smooth(), name="smooth")
smooth.iterables = ("fwhm", fwhm)
# Gunzip - unzip functional
gunzip = Node(Gunzip(), name="gunzip")
gunzip2 = Node(Gunzip(), name="gunzip2")


## Now we create our preprocessing workflow which defines the order of the steps and connecting the nodes.

In [15]:
# Connect all components of the preprocessing workflow
# Connect SelectFiles and DataSink to the workflow
preproc.connect([(infosource, selectfiles, [('subject_id', 'subject_id')]),
                 (selectfiles, gunzip, [('func', 'in_file')]),
                 (selectfiles, gunzip2, [('anat', 'in_file')]),
                 (realign, datasink, [('mean_image', 'realign.@mean'),
                                      ('realignment_parameters',
                                       'realign.@parameters'),
                                      ]),
                 (norm12, datasink, [('normalized_files','normalized.@files')]),
                 (st, datasink, [('timecorrected_files','timecorrected.@files')]),
                 (smooth, datasink, [('smoothed_files', 'smooth')]),
                 
                 ])

## We can save this order in a graph which makes it more vividly. Here in two different ways.

In [13]:
preproc.write_graph(graph2use="colored")

210720-16:15:09,869 nipype.workflow INFO:
	 Generated workflow graph: /media/jt/Daten1/preproc/workingdir/preproc/graph.png (graph2use=colored, simple_form=True).


'/media/jt/Daten1/preproc/workingdir/preproc/graph.png'

In [7]:
preproc.write_graph(graph2use='flat')

210720-16:14:42,196 nipype.workflow INFO:
	 Generated workflow graph: /media/jt/Daten1/preproc/workingdir/preproc/graph.png (graph2use=flat, simple_form=True).


'/media/jt/Daten1/preproc/workingdir/preproc/graph.png'

## Now we define our input and output stream.

In [14]:
# Infosource - a function free node to iterate over the list of subject names
infosource = Node(IdentityInterface(fields=['subject_id', 'task_name']),
                  name="infosource")
infosource.iterables = [('subject_id', subject_list),
                        ('task_name', task_list)]

# SelectFiles - to grab the data (alternativ to DataGrabber)
anat_file = opj('sub-{subject_id}', 'anat', 'sub-{subject_id}_T1w.nii.gz')
func_file = opj('sub-{subject_id}', 'func', 'sub-{subject_id}_task-rest_bold.nii.gz')

templates = {'anat': anat_file,
             'func': func_file}
selectfiles = Node(SelectFiles(templates,
                               base_directory='/home/jt/ds002748/'),
                   name="selectfiles")

# Datasink - creates output folder for important outputs
datasink = Node(DataSink(base_directory=experiment_dir,
                         container=output_dir),
                name="datasink")

## Use the following DataSink output substitutions
substitutions = [('_subject_id_', 'sub-'),
                 ('_task_name_', '/task-'),
                 ('_fwhm_', 'fwhm-'),
                 ('_roi', ''),
                 ('_mcf', ''),
                 ('_st', ''),
                 ('_flirt', ''),
                 ('.nii_mean_reg', '_mean'),
                 ('.nii.par', '.par'),
                 ]
subjFolders = [('fwhm-%s/' % f, 'fwhm-%s_' % f) for f in fwhm]
substitutions.extend(subjFolders)
datasink.inputs.substitutions = substitutions

In [16]:
preproc.write_graph(graph2use='flat')

210720-16:15:16,604 nipype.workflow INFO:
	 Generated workflow graph: /media/jt/Daten1/preproc/workingdir/preproc/graph.png (graph2use=flat, simple_form=True).


'/media/jt/Daten1/preproc/workingdir/preproc/graph.png'

## And now it's time to run all the stuff. Have fun, may take some time....

In [17]:
preproc.run('MultiProc', plugin_args={'n_procs': 6})

210720-16:15:22,120 nipype.workflow INFO:
	 Workflow preproc settings: ['check', 'execution', 'logging', 'monitoring']
210720-16:15:22,502 nipype.workflow INFO:
	 Running in parallel.
210720-16:15:22,515 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 72 jobs ready. Free memory (GB): 13.80/13.80, Free processors: 6/6.
210720-16:15:22,584 nipype.workflow INFO:
	 [Node] Setting-up "preproc.selectfiles" in "/media/jt/Daten1/preproc/workingdir/preproc/_subject_id_72_task_name_restingstatewithclosedeyes/selectfiles".
210720-16:15:22,586 nipype.workflow INFO:
	 [Node] Setting-up "preproc.selectfiles" in "/media/jt/Daten1/preproc/workingdir/preproc/_subject_id_70_task_name_restingstatewithclosedeyes/selectfiles".
210720-16:15:22,586 nipype.workflow INFO:
	 [Node] Setting-up "preproc.selectfiles" in "/media/jt/Daten1/preproc/workingdir/preproc/_subject_id_71_task_name_restingstatewithclosedeyes/selectfiles".
210720-16:15:22,592 nipype.workflow INFO:
	 [Node] Setting-up "preproc.select

210720-16:15:28,519 nipype.workflow INFO:
	 [Job 25] Completed (preproc.gunzip2).
210720-16:15:28,521 nipype.workflow INFO:
	 [Job 26] Completed (preproc.gunzip).
210720-16:15:28,523 nipype.workflow INFO:
	 [Job 33] Completed (preproc.gunzip2).
210720-16:15:28,527 nipype.workflow INFO:
	 [MultiProc] Running 3 tasks, and 70 jobs ready. Free memory (GB): 13.20/13.80, Free processors: 3/6.
                     Currently running:
                       * preproc.realign
                       * preproc.realign
                       * preproc.realign
210720-16:15:28,573 nipype.workflow INFO:
	 [Node] Setting-up "preproc.realign" in "/media/jt/Daten1/preproc/workingdir/preproc/_subject_id_69_task_name_restingstatewithclosedeyes/realign".
210720-16:15:28,573 nipype.workflow INFO:
	 [Node] Setting-up "preproc.gunzip" in "/media/jt/Daten1/preproc/workingdir/preproc/_subject_id_68_task_name_restingstatewithclosedeyes/gunzip".
210720-16:15:28,574 nipype.workflow INFO:
	 [Node] Setting-up "prepro

210720-16:18:07,544 nipype.workflow INFO:
	 [Node] Finished "preproc.slicetiming".
210720-16:18:08,666 nipype.workflow INFO:
	 [Job 4] Completed (preproc.slicetiming).
210720-16:18:08,671 nipype.workflow INFO:
	 [MultiProc] Running 5 tasks, and 67 jobs ready. Free memory (GB): 12.80/13.80, Free processors: 1/6.
                     Currently running:
                       * preproc.slicetiming
                       * preproc.slicetiming
                       * preproc.slicetiming
                       * preproc.slicetiming
                       * preproc.slicetiming
210720-16:18:08,724 nipype.workflow INFO:
	 [Node] Setting-up "preproc.normalise" in "/media/jt/Daten1/preproc/workingdir/preproc/_subject_id_72_task_name_restingstatewithclosedeyes/normalise".
210720-16:18:09,355 nipype.workflow INFO:
	 [Node] Finished "preproc.slicetiming".
210720-16:18:09,359 nipype.workflow INFO:
	 [Node] Running "normalise" ("nipype.interfaces.spm.preprocess.Normalize12")
210720-16:18:10,572 nipyp

210720-16:23:02,596 nipype.workflow INFO:
	 [Node] Finished "preproc.normalise".
210720-16:23:02,941 nipype.workflow INFO:
	 [Job 13] Completed (preproc.normalise).
210720-16:23:02,947 nipype.workflow INFO:
	 [MultiProc] Running 5 tasks, and 67 jobs ready. Free memory (GB): 12.80/13.80, Free processors: 1/6.
                     Currently running:
                       * preproc.smooth
                       * preproc.smooth
                       * preproc.smooth
                       * preproc.normalise
                       * preproc.normalise
210720-16:23:03,9 nipype.workflow INFO:
	 [Node] Setting-up "preproc.smooth" in "/media/jt/Daten1/preproc/workingdir/preproc/_subject_id_71_task_name_restingstatewithclosedeyes/_fwhm_8/smooth".
210720-16:23:03,23 nipype.workflow INFO:
	 [Node] Running "smooth" ("nipype.interfaces.spm.preprocess.Smooth")
210720-16:23:03,629 nipype.workflow INFO:
	 [Node] Finished "preproc.normalise".
210720-16:23:04,944 nipype.workflow INFO:
	 [Job 29] Compl

210720-16:24:00,575 nipype.workflow INFO:
	 [Node] Finished "preproc.gunzip".
210720-16:24:00,994 nipype.workflow INFO:
	 [Job 50] Completed (preproc.gunzip).
210720-16:24:01,0 nipype.workflow INFO:
	 [MultiProc] Running 5 tasks, and 66 jobs ready. Free memory (GB): 12.80/13.80, Free processors: 1/6.
                     Currently running:
                       * preproc.smooth
                       * preproc.smooth
                       * preproc.smooth
                       * preproc.smooth
                       * preproc.smooth
210720-16:24:01,77 nipype.workflow INFO:
	 [Node] Setting-up "preproc.realign" in "/media/jt/Daten1/preproc/workingdir/preproc/_subject_id_66_task_name_restingstatewithclosedeyes/realign".
210720-16:24:01,770 nipype.workflow INFO:
	 [Node] Running "realign" ("nipype.interfaces.spm.preprocess.Realign")
210720-16:24:03,1 nipype.workflow INFO:
	 [MultiProc] Running 6 tasks, and 65 jobs ready. Free memory (GB): 12.60/13.80, Free processors: 0/6.
            

...

210720-17:52:58,168 nipype.workflow INFO:
	 [Node] Running "smooth" ("nipype.interfaces.spm.preprocess.Smooth")
210720-17:53:00,57 nipype.workflow INFO:
	 [MultiProc] Running 6 tasks, and 2 jobs ready. Free memory (GB): 12.60/13.80, Free processors: 0/6.
                     Currently running:
                       * preproc.smooth
                       * preproc.smooth
                       * preproc.normalise
                       * preproc.normalise
                       * preproc.normalise
                       * preproc.normalise
210720-17:53:29,500 nipype.workflow INFO:
	 [Node] Finished "preproc.smooth".
210720-17:53:30,83 nipype.workflow INFO:
	 [Job 518] Completed (preproc.smooth).
210720-17:53:30,89 nipype.workflow INFO:
	 [MultiProc] Running 5 tasks, and 3 jobs ready. Free memory (GB): 12.80/13.80, Free processors: 1/6.
                     Currently running:
                       * preproc.smooth
                       * preproc.normalise
                       * pre

210720-17:54:18,373 nipype.interface INFO:
	 sub: /media/jt/Daten1/preproc/datasink/timecorrected/_subject_id_07_task_name_restingstatewithclosedeyes/arsub-07_task-rest_bold.nii -> /media/jt/Daten1/preproc/datasink/timecorrected/sub-07/task-restingstatewithclosedeyes/arsub-07_task-rest_bold.nii
210720-17:54:18,381 nipype.interface INFO:
	 sub: /media/jt/Daten1/preproc/datasink/smooth/_subject_id_07_task_name_restingstatewithclosedeyes/_fwhm_8/swarsub-07_task-rest_bold.nii -> /media/jt/Daten1/preproc/datasink/smooth/sub-07/task-restingstatewithclosedeyes/fwhm-8_swarsub-07_task-rest_bold.nii
210720-17:54:18,399 nipype.workflow INFO:
	 [Node] Finished "preproc.datasink".
210720-17:54:20,132 nipype.workflow INFO:
	 [Job 527] Completed (preproc.datasink).
210720-17:54:20,138 nipype.workflow INFO:
	 [MultiProc] Running 5 tasks, and 1 jobs ready. Free memory (GB): 12.80/13.80, Free processors: 1/6.
                     Currently running:
                       * preproc.realign
              

210720-17:56:00,292 nipype.workflow INFO:
	 [Node] Finished "preproc.normalise".
210720-17:56:02,232 nipype.workflow INFO:
	 [Job 541] Completed (preproc.normalise).
210720-17:56:02,238 nipype.workflow INFO:
	 [MultiProc] Running 5 tasks, and 1 jobs ready. Free memory (GB): 12.80/13.80, Free processors: 1/6.
                     Currently running:
                       * preproc.slicetiming
                       * preproc.smooth
                       * preproc.slicetiming
                       * preproc.normalise
                       * preproc.normalise
210720-17:56:02,323 nipype.workflow INFO:
	 [Node] Setting-up "preproc.smooth" in "/media/jt/Daten1/preproc/workingdir/preproc/_subject_id_05_task_name_restingstatewithclosedeyes/_fwhm_8/smooth".
210720-17:56:02,335 nipype.workflow INFO:
	 [Node] Running "smooth" ("nipype.interfaces.spm.preprocess.Smooth")
210720-17:56:04,237 nipype.workflow INFO:
	 [MultiProc] Running 6 tasks, and 0 jobs ready. Free memory (GB): 12.60/13.80, Free

210720-17:57:13,841 nipype.workflow INFO:
	 [Node] Finished "preproc.smooth".
210720-17:57:14,299 nipype.workflow INFO:
	 [Job 542] Completed (preproc.smooth).
210720-17:57:14,306 nipype.workflow INFO:
	 [MultiProc] Running 4 tasks, and 1 jobs ready. Free memory (GB): 13.00/13.80, Free processors: 2/6.
                     Currently running:
                       * preproc.normalise
                       * preproc.smooth
                       * preproc.smooth
                       * preproc.normalise
210720-17:57:14,381 nipype.workflow INFO:
	 [Node] Setting-up "preproc.datasink" in "/media/jt/Daten1/preproc/workingdir/preproc/_subject_id_05_task_name_restingstatewithclosedeyes/_fwhm_8/datasink".
210720-17:57:14,409 nipype.workflow INFO:
	 [Node] Running "datasink" ("nipype.interfaces.io.DataSink")
210720-17:57:14,412 nipype.interface INFO:
	 sub: /media/jt/Daten1/preproc/datasink/realign/_subject_id_05_task_name_restingstatewithclosedeyes/meansub-05_task-rest_bold.nii -> /media/jt

210720-17:59:26,427 nipype.workflow INFO:
	 [MultiProc] Running 2 tasks, and 0 jobs ready. Free memory (GB): 13.40/13.80, Free processors: 4/6.
                     Currently running:
                       * preproc.smooth
                       * preproc.normalise
210720-17:59:43,968 nipype.workflow INFO:
	 [Node] Finished "preproc.normalise".
210720-17:59:44,444 nipype.workflow INFO:
	 [Job 573] Completed (preproc.normalise).
210720-17:59:44,446 nipype.workflow INFO:
	 [MultiProc] Running 1 tasks, and 1 jobs ready. Free memory (GB): 13.60/13.80, Free processors: 5/6.
                     Currently running:
                       * preproc.smooth
210720-17:59:44,489 nipype.workflow INFO:
	 [Node] Setting-up "preproc.smooth" in "/media/jt/Daten1/preproc/workingdir/preproc/_subject_id_01_task_name_restingstatewithclosedeyes/_fwhm_8/smooth".
210720-17:59:44,499 nipype.workflow INFO:
	 [Node] Running "smooth" ("nipype.interfaces.spm.preprocess.Smooth")
210720-17:59:46,447 nipype.workflow

## In case of crash you can load the crash report here and check for further information.

In [175]:
%%bash
nipypecli crash /home/jt/Downloads/crash-20210719-131157-jt-normalise.b0-eff9db7a-d3c1-4b0a-86e1-254c3b213aaf.pklz



File: /home/jt/Downloads/crash-20210719-131157-jt-normalise.b0-eff9db7a-d3c1-4b0a-86e1-254c3b213aaf.pklz
Node: preproc.normalise
Working directory: /home/jt/ds002748/derivatives/preproc/workingdir/preproc/_subject_id_03_task_name_restingstatewithclosedeyes/normalise


Node inputs:

affine_regularization_type = <undefined>
apply_to_files = ['/home/jt/ds002748/derivatives/preproc/workingdir/preproc/_subject_id_03_task_name_restingstatewithclosedeyes/slicetiming/arsub-03_task-rest_bold.nii']
bias_fwhm = <undefined>
bias_regularization = <undefined>
deformation_file = y_*.nii
image_to_align = <undefined>
jobtype = estwrite
matlab_cmd = /home/jt/temp/spm12_latest/spm12/run_spm12.sh /home/jt/Matlab/v98 script
mfile = True
out_prefix = w
paths = <undefined>
sampling_distance = <undefined>
smoothness = <undefined>
tpm = <undefined>
use_mcr = True
use_v8struct = True
warping_regularization = <undefined>
write_bounding_box = <undefined>
write_interp = <undefined>
write_voxel_sizes = <undefined